In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sastrawi.stopword import StopWordRemover
from sastrawi.stemmer import Stemmer

In [ ]:
# inisialisasi stopwordremover dan stemmer
stopword_remover = StopWordRemover()
stemmer = Stemmer()

def preprocess_text(text):
    #menhapus stopword
    text1_no_stopwords = stopword_remover.remove(text)
    #melakukan stemming
    text_stemmed = stemmer.stem(text_no_stopwords)
    return text_stemmed

In [ ]:
#muat data file excel
data = pd.read_excel('text_data.xlsx')

#pra-pemrosesan teks
data['text'] = data['text'].apply(preprocess_text)

# misalnya data memiliki kolom 'text' untuk teks dan 'sentiment' untuk label sentimen
texts = data['text']
sentiments = data['sentiment']

#split data menjadi data pelatihan dan data uji
x_train, x_test, y_train, y_test = train_test_split(texts, sentiments, test_size=0.3, random_state=42)

In [ ]:
# inisialisasi countVectorizer
vectorizer = CountVectorizer()

# fit dan transform data pelatihan
x_train_counts = vectorizer.fit.transform(x_train)

# transform data uji
x_test_counts = vectorizer.transform(x_test)

# inisialisasi multinomialNB
model = MultinomialNB()

# latih model
model.fit(x_train_counts, y_train)

# prediksi sentimen untuk data uji
predictions = model.predict(x_test_counts)

In [ ]:
# menampilkan hasil evaluasi
print("evaluasi model:")
print(classification_report(y_test, predictions))

#menampilkan beberapa hasil prediksi
print("\nBeberapa hasil prediksi:")
for text, sentiment in zip(x_test, predictions):
    print(f'teks: {text}\nPrediksi Sentiment: {sentiment}\n')

# ekspor hasil prediksi ke file excel
results = pd.DataFrame({'text': x_test, 'predicted_sentiment': predictions})
results.to_excel('prediction.xlsx', index=False)

print("Hasil prediksi telah diekspor ke 'predictions.xlsx'")

In [ ]:
# fungsi untuk menganalisis teks baru
def analyze_text(text):
    text_counts = vectorizer.transform([text])
    predictions = model.predict(text_counts)[0]
    return predictions

# menganalisis teks dari input pengguna
while True:
    user_input = input("\nMasukkan teks untuk analisis sentimen (atau ketik 'exit' untuk keluar): ")
    if user_input.lower() == 'exit':
        break
    sentiment = analyze_text(user_input)
    print(f'Prediksi sentimen untuk "{user_input}": {sentiment}')